In [2]:
from neo4j import GraphDatabase
from dotenv import load_dotenv
import os

In [3]:
load_dotenv()
URI = os.environ.get('NEO4J_URI')
USERNAME = os.environ.get('NEO4J_USERNAME')
PASSWORD = os.environ.get('NEO4J_PASSWORD')

In [4]:
driver = GraphDatabase.driver(URI, auth=(USERNAME, PASSWORD))

In [5]:
def run_query(query, params={}):
    with driver.session() as session:
        result = session.run(query, params)
        return result.data()  # Returns list of dictionaries


In [6]:
#First query
query1="""
    MATCH (p:paper)-[:presented]->(ce:ConferenceEdition) 
    OPTIONAL MATCH (p)<-[c:cite]-(:paper) 
    WITH ce, p, COUNT(c) AS cnt 
    ORDER BY ce, cnt DESC
    WITH ce, COLLECT({paper: p, cite_nb: cnt}) AS paper_cnt 
    RETURN ce.confwsEditionId AS confId, paper_cnt[..3] AS top3
"""

results = run_query(query1)
for row in results:
    print(row['confId'])  # Print confId first
    for paper in row['top3']:  
        print(paper['paper']['title'], paper['cite_nb']) 
    print("------------------------------------------")

Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.AggregationSkippedNull} {category: UNRECOGNIZED} {title: The query contains an aggregation function that skips null values.} {description: null value eliminated in set function.} {position: None} for query: '\n    MATCH (p:paper)-[:presented]->(ce:ConferenceEdition) \n    OPTIONAL MATCH (p)<-[c:cite]-(:paper) \n    WITH ce, p, COUNT(c) AS cnt \n    ORDER BY ce, cnt DESC\n    WITH ce, COLLECT({paper: p, cite_nb: cnt}) AS paper_cnt \n    RETURN ce.confwsEditionId AS confId, paper_cnt[..3] AS top3\n'


768b87bb-8a18-4d9c-a161-4d483c776bcf2021
Robust Point Cloud Registration Framework Based on Deep Graph Matching 1
I3Net: Implicit Instance-Invariant Network for Adapting One-Stage Object Detectors 1
Masked Autoencoders Are Scalable Vision Learners 1
------------------------------------------
3f2626a8-9d78-42ca-9e0d-4b853b59cdcc2020
Robotic Grasping of Fully-Occluded Objects using RF Perception 1
Semantic Linking Maps for Active Visual Object Search 1
Semantic and Geometric Modeling with Neural Message Passing in 3D Scene Graphs for Hierarchical Mechanical Search 0
------------------------------------------
a0edb93b-1e95-4128-a295-6b1659149cef2024
ZeroG: Investigating Cross-dataset Zero-shot Transferability in Graphs 0
------------------------------------------
d2e111a6-dd60-474c-ae8a-2b5773cca07d2023
Automatic Program Repair via Learning Edits on Sequence Code Property Graph 0
------------------------------------------
768b87bb-8a18-4d9c-a161-4d483c776bcf2020
Semi-Supervised Semantic S

In [10]:
query2="""
MATCH (a:author)-[:writes]->(p:paper)-[:presented]->(ce:ConferenceEdition)-[:has_edition]->(c:Conference)
WITH a, c, COLLECT(DISTINCT ce) AS edi
WHERE SIZE(edi) >= 2
RETURN a.authorName AS author, c.name AS conference, SIZE(edi) AS editions
ORDER BY editions DESC"""


results = run_query(query2)
for row in results:
    print(row)

{'author': 'M. Nießner', 'conference': 'Computer Vision and Pattern Recognition', 'editions': 3}
{'author': 'Angela Dai', 'conference': 'Computer Vision and Pattern Recognition', 'editions': 3}
{'author': 'Hanwang Zhang', 'conference': 'Computer Vision and Pattern Recognition', 'editions': 2}


In [ ]:
query3="""
//with date().year as currentYear
WITH 2020 AS currentYear
MATCH (j:journal) <-[:published]- (p:paper)
WHERE date(p.publicationDate).year IN [currentYear - 1, currentYear - 2]
WITH j, COUNT(p) AS citableItems, COLLECT(p) AS citablePapers, currentYear

UNWIND citablePapers AS p
MATCH (p) <-[:cite]- (cp:paper)
WHERE date(cp.publicationDate).year = currentYear

RETURN j.journalName,
       citableItems,
       COUNT(cp) AS citations,
       COUNT(cp) * 1.0 / citableItems AS impact_factor
"""

results = run_query(query3)
for row in results:
    print(row)

In [ ]:
query4="""
MATCH (a:author) -[:writes]-> (p:paper) <-[:cite]- (cp:paper)
WITH a, p, COUNT(cp) AS citations
ORDER BY citations desc
WITH a, COLLECT(citations) as citationList

UNWIND RANGE(1, SIZE(citationList)) as h
WITH a, h, citationList[h-1] as citationCount
WHERE citationCount >= h
RETURN a.authorName, MAX(h) AS h_index
"""

results = run_query(query4)
for row in results:
    print(row)